Doc : https://doc.demarches-simplifiees.fr/pour-aller-plus-loin/graphql et https://demarches-simplifiees-graphql.netlify.app/query.doc.html

In [1]:
import requests
import json
import pandas as pd
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
import configparser
config = configparser.ConfigParser()
config.read('./secret.ini')

['./secret.ini']

In [2]:
arr = []
endCursor = ''
hasNextPage = True
while(hasNextPage):
    query = """query{
        demarche(number: 30928) {
            id
            dossiers(state:accepte after:\""""+endCursor+"""\") { 
                nodes {
                    champs {
                        label
                        stringValue

                    }
                    annotations {
                        label
                        stringValue
                    }
                    number
                    id
                    state
                    demandeur { 
                        ... on PersonnePhysique { 
                            civilite
                            nom
                            prenom
                        } 
                        ... on PersonneMorale { 
                            siret
                            codePostal
                            naf
                        }
                    }
                    groupeInstructeur{
                        label
                    }
                }

                  pageInfo {
                    hasNextPage
                    endCursor
                  }
            }
        } 
    }"""
    headers = {"Authorization": "Bearer "+config['DS']['bearer']}
    url = 'https://www.demarches-simplifiees.fr/api/v2/graphql'
    r = requests.post(url, headers=headers, json={'query': query})
    print(r.status_code)
    data = r.json()
    hasNextPage = data['data']['demarche']['dossiers']['pageInfo']['hasNextPage']
    endCursor = data['data']['demarche']['dossiers']['pageInfo']['endCursor']
    for d in data['data']['demarche']['dossiers']['nodes']:
        mydict = {}
        mydict['number'] = d['number']
        mydict['id'] = d['id']
        mydict['state'] = d['state']
        mydict['siret'] = d['demandeur']['siret']
        mydict['codePostal'] = d['demandeur']['codePostal']
        mydict['naf'] = d['demandeur']['naf']

        mydict['groupe_instructeur'] = d['groupeInstructeur']['label']

        for c in d['champs']:
            mydict[c['label']] = c['stringValue']
        for a in d['annotations']:
            mydict[a['label']] = a['stringValue']
        arr.append(mydict)

200
200


In [3]:
df = pd.DataFrame(arr)

In [4]:
df.shape

(110, 160)

In [5]:
df.state.value_counts()

accepte    110
Name: state, dtype: int64

In [6]:
dffinal = df[['id','siret','naf','state',
              'Montant total du prêt demandé',
              'Montant proposé','Durée du prêt','Quelle forme prend l\'aide ?',
              'codePostal','Quels sont vos effectifs ?',
              'groupe_instructeur','Département']]

In [7]:
dffinal = dffinal.rename(columns={
    'naf':'code_naf',
    'state':'statut',
    'Montant total du prêt demandé':'montant_demande',
    'Montant proposé':'montant',
    'Quelle forme prend l\'aide ?':'type_aide',
    'Durée du prêt':'duree',
    'Quels sont vos effectifs ?':'effectifs',
    'Département':'departement'
})

In [8]:
dffinal.effectifs = dffinal.effectifs.astype(float)

In [9]:
dffinal[dffinal['effectifs'] < 0]

,id,siret,code_naf,statut,montant_demande,montant,duree,type_aide,codePostal,effectifs,groupe_instructeur,departement
25,RG9zc2llci0yMDQ3MjU4,50226772700019,3832Z,accepte,2034000,800000,6,Prêt à taux bonifié,38570,-18.0,Auvergne-Rhône-Alpes,38 - Isère
102,RG9zc2llci0zMjA1MDE5,31176543200043,2932Z,accepte,-500000,250000,10,Avance remboursable,18230,-32.0,Centre-Val de Loire,18 - Cher


In [10]:
dffinal.montant = dffinal.montant.astype(float)

In [11]:
dffinal.montant.sum()

78921407.0

In [12]:
dffinal['dep'] = dffinal['departement'].apply(lambda x: x.split(" - ")[0])

In [13]:
import time

In [14]:
siret_for_api = dffinal.siret.unique()

In [15]:
# Pour chaque SIRET on appelle l'API entreprise pour voir si on récupère des informations
# au niveau du SIRET
arr = []
i = 0
for siret in siret_for_api:
    # On ne doit pas surcharger l'API donc on met un temps entre chaque requête (il y a un quota à ne pas dépasser)
    # C'est un peu long...
    time.sleep(0.3)
    i = i + 1
    if(i%10 == 0):
        print(str(i))
    row = {}
    url = "https://entreprise.api.gouv.fr/v2/effectifs_annuels_acoss_covid/"+siret[:9]+"?non_diffusables=true&recipient=13001653800014&object=dgefp&context=%22dashboard%20aides%20aux%20entreprises%22&token="+config['API']['token']
    try:
        r = requests.get(url=url)
        try:
            mydict = r.json()
        except:
            pass
        if "effectifs_annuels" in mydict:
            arr.append(mydict)
    except requests.exceptions.ConnectionError:
        print("Error : "+siret)

10
20
30
40
50
60
70
80
90
100
110


In [16]:
dfapi = pd.DataFrame(arr)

In [17]:
dfapi = dfapi.drop_duplicates(subset=['siren'], keep='first')

In [18]:
dffinal['siren'] = dffinal['siret'].apply(lambda x: str(x)[:9])

In [19]:
dffinal = pd.merge(dffinal,dfapi,on='siren',how='left')

In [20]:
dffinal['effectifs_annuels'] = dffinal['effectifs_annuels'].apply(lambda x: int(float(x)) if x == x else x)

In [21]:
dffinal.loc[dffinal.effectifs_annuels.isna(),'effectifs_annuels']=dffinal['effectifs']

In [22]:
reg = pd.read_csv("https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/utils/region2019.csv",dtype=str)
dep = pd.read_csv("https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/utils/departement2019.csv",dtype=str)

In [23]:
dep = dep[['dep','reg']]
reg = reg[['reg','libelle']]

In [24]:
dep = pd.merge(dep,reg,on='reg',how='left')

In [25]:
dffinal = pd.merge(dffinal,dep,on='dep',how='left')

In [26]:
dffinal = dffinal[['montant','effectifs_annuels','reg','libelle','type_aide']]

In [27]:
dffinal = dffinal.rename(columns={'effectifs_annuels':'effectifs'})

In [28]:
dffinal.effectifs = dffinal.effectifs.astype(float)
dffinal.montant = dffinal.montant.astype(float)

In [29]:
arr = []
mydict = {}
mydict['type'] = []

mydict2 = {}
mydict2['libelle'] = 'Prêt à taux bonifié'
mydict2['nombre'] = str(dffinal[dffinal['type_aide'] == 'Prêt à taux bonifié'].shape[0])
mydict2['montant'] = str(dffinal[dffinal['type_aide'] == 'Prêt à taux bonifié'].montant.sum())
mydict2['effectifs'] = str(dffinal[dffinal['type_aide'] == 'Prêt à taux bonifié'].effectifs.sum())
mydict['type'].append(mydict2)

mydict2 = {}
mydict2['libelle'] = 'Avance remboursable'
mydict2['nombre'] = str(dffinal[dffinal['type_aide'] == 'Avance remboursable'].shape[0])
mydict2['montant'] = str(dffinal[dffinal['type_aide'] == 'Avance remboursable'].montant.sum())
mydict2['effectifs'] = str(dffinal[dffinal['type_aide'] == 'Avance remboursable'].effectifs.sum())
mydict['type'].append(mydict2)

mydict['nombre'] = str(dffinal.shape[0])
mydict['montant'] = str(dffinal.montant.sum())
mydict['effectifs'] = str(dffinal.effectifs.sum())

arr.append(mydict)

In [30]:
arr

[{'type': [{'libelle': 'Prêt à taux bonifié',
    'nombre': '86',
    'montant': '67305607.0',
    'effectifs': '6432.0'},
   {'libelle': 'Avance remboursable',
    'nombre': '24',
    'montant': '11615800.0',
    'effectifs': '833.0'}],
  'nombre': '110',
  'montant': '78921407.0',
  'effectifs': '7265.0'}]

In [31]:
with open('arpb-maille-national.json', 'w') as outfile:
            json.dump(arr, outfile)

In [32]:
dffinal[dffinal['type_aide'] == 'Prêt à taux bonifié'].effectifs.sum()

6432.0

In [33]:
arr = []
for r in reg.reg.unique():
    if(dffinal[dffinal['reg'] == r].shape[0] > 0):
    
        mydict = {}
        mydict['type'] = []

        mydict2 = {}
        mydict2['libelle'] = 'Prêt à taux bonifié'
        mydict2['nombre'] = str(dffinal[(dffinal['type_aide'] == 'Prêt à taux bonifié') & (dffinal['reg'] == r)].shape[0])
        mydict2['montant'] = str(dffinal[(dffinal['type_aide'] == 'Prêt à taux bonifié') & (dffinal['reg'] == r)].montant.sum())
        mydict2['effectifs'] = str(dffinal[(dffinal['type_aide'] == 'Prêt à taux bonifié') & (dffinal['reg'] == r)].effectifs.sum())
        mydict['type'].append(mydict2)

        mydict2 = {}
        mydict2['libelle'] = 'Avance remboursable'
        mydict2['nombre'] = str(dffinal[(dffinal['type_aide'] == 'Avance remboursable') & (dffinal['reg'] == r)].shape[0])
        mydict2['montant'] = str(dffinal[(dffinal['type_aide'] == 'Avance remboursable') & (dffinal['reg'] == r)].montant.sum())
        mydict2['effectifs'] = str(dffinal[(dffinal['type_aide'] == 'Avance remboursable') & (dffinal['reg'] == r)].effectifs.sum())
        mydict['type'].append(mydict2)

        mydict['nombre'] = str(dffinal[dffinal['reg'] == r].shape[0])
        mydict['montant'] = str(dffinal[dffinal['reg'] == r].montant.sum())
        mydict['effectifs'] = str(dffinal[dffinal['reg'] == r].effectifs.sum())

    else:
        mydict = {}
        mydict['type'] = None
        mydict['nombre'] = None
        mydict['montant'] = None
        mydict['effectifs'] = None
    mydict['reg'] = r
    mydict['libelle'] = reg[reg['reg'] == r].iloc[0]['libelle']
    arr.append(mydict)

In [34]:
arr

[{'type': None,
  'nombre': None,
  'montant': None,
  'effectifs': None,
  'reg': '01',
  'libelle': 'Guadeloupe'},
 {'type': None,
  'nombre': None,
  'montant': None,
  'effectifs': None,
  'reg': '02',
  'libelle': 'Martinique'},
 {'type': None,
  'nombre': None,
  'montant': None,
  'effectifs': None,
  'reg': '03',
  'libelle': 'Guyane'},
 {'type': [{'libelle': 'Prêt à taux bonifié',
    'nombre': '0',
    'montant': '0.0',
    'effectifs': '0.0'},
   {'libelle': 'Avance remboursable',
    'nombre': '4',
    'montant': '1140000.0',
    'effectifs': '37.0'}],
  'nombre': '4',
  'montant': '1140000.0',
  'effectifs': '37.0',
  'reg': '04',
  'libelle': 'La Réunion'},
 {'type': None,
  'nombre': None,
  'montant': None,
  'effectifs': None,
  'reg': '06',
  'libelle': 'Mayotte'},
 {'type': [{'libelle': 'Prêt à taux bonifié',
    'nombre': '10',
    'montant': '6115000.0',
    'effectifs': '855.0'},
   {'libelle': 'Avance remboursable',
    'nombre': '2',
    'montant': '1100000.0',


In [35]:
with open('arpb-maille-regional.json', 'w') as outfile:
            json.dump(arr, outfile)

In [36]:
arr = []
with open('arpb-maille-departemental.json', 'w') as outfile:
            json.dump(arr, outfile)

In [37]:
dfgb = dffinal.groupby(['reg','libelle','type_aide'],as_index=False).sum()

In [38]:
dfgb.to_csv("prets-directs-etat-regional.csv",index=False)

In [39]:
dfgb = dfgb.groupby(['type_aide'],as_index=False).sum()

In [40]:
dfgb.to_csv("prets-directs-etat-national.csv",index=False)

In [41]:
dfgb

,type_aide,montant,effectifs
0,Avance remboursable,11615800.0,833.0
1,Prêt à taux bonifié,67305607.0,6432.0
